# 空氣污染監測網 網路爬蟲實作練習


* 能夠利用 selenium + BeautifulSoup 撰寫爬蟲，並存放到合適的資料結構


## 作業目標

根據範例 ，完成以下問題：

* ① 取出 台北市士林區 2018/01 – 2018/08 的 SO2 資料
* ② 取出 台北市士林區 2018/01 – 2018/08 的 SO2、CO 資料





### ① 取出 台北市士林區 2018/01 – 2018/08 的 SO2 資料

In [1]:
# 1.打開瀏覽器
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
browser = webdriver.Chrome(executable_path='chromedriver')
browser.get("http://taqm.epa.gov.tw/taqm/tw/MonthlyAverage.aspx")
    # 模擬使用者操作行為，選擇/點擊
selectSite = Select(browser.find_element_by_id("ctl05_ddlSite"))
selectSite.select_by_value('11')
selectYear = Select(browser.find_element_by_id("ctl05_ddlYear"))
selectYear.select_by_value('2018')
browser.find_element_by_id('ctl05_btnQuery').click()
# 2.取得資料（等瀏覽器資料出現後才執行）：丟到 BeautifulSoup 解析
import time
time.sleep(10)

html_source = browser.page_source
soup = BeautifulSoup(html_source, "html5lib")
table = soup.find('table', class_='TABLE_G')
# 3.關閉瀏覽器
browser.quit()
# 4.表格內容轉為dict結構
dict_all = {}
for tr in table.find_all('tr')[1:]:     # column name不看，每個tr就是一監測項目
    for i, td in enumerate(tr.find_all('td')):
        if len(tr.find_all('td')) == 5:      # tr內有5個td的是項目名稱 + 第一個月資料
            if i == 0:     # 第一個td是項目名稱
                obs = td.text
                dict_all.setdefault(obs, {})      # {'項目名稱': {}}
            if i == 2:     # 第一個月日期
                date = td.text
            if i == 3:     # 第一個月數值
                value = td.text
                dict_all[obs][date] = value     # 字典裡面還是字典，進行新增資料
        if len(tr.find_all('td')) == 3:      # tr內有3個td的是 第二個月之後資料
            if i == 0:
                date = td.text
            if i == 1:
                value = td.text
                dict_all[obs][date] = value
# 5.pandas表示
import pandas as pd
all_data = pd.DataFrame(dict_all)[:-1]
all_data = pd.DataFrame(dict_all)[:-1]
print(all_data[:8]['SO2'])

2018/01    1.80
2018/02    1.90
2018/03    2.20
2018/04    2.30
2018/05    3.10
2018/06    2.70
2018/07    2.20
2018/08    2.40
Name: SO2, dtype: object


### ② 取出 台北市士林區 2018/01 – 2018/08 的 SO2、CO 資料

In [2]:
print(all_data[:8][['SO2', 'CO']])

          SO2    CO
2018/01  1.80  0.34
2018/02  1.90  0.44
2018/03  2.20  0.40
2018/04  2.30  0.38
2018/05  3.10  0.34
2018/06  2.70  0.29
2018/07  2.20  0.21
2018/08  2.40  0.30
